# Division del Dataset
En este notebook se muestran algunos de los mecanismos mas utilizados para la division del datset 
#### DataSet 
#### Descripcion
We apologize, this dataset is no longer available.

ISCX NSL-KDD is a data set suggested to solve some of the inherent problems of the KDD'99 data set which are mentioned in [1]. Although, this new version of the KDD data set still suffers from some of the problems discussed by McHugh and may not be a perfect representative of existing real networks, because of the lack of public data sets for network-based IDSs, we believe it still can be applied as an effective benchmark data set to help researchers compare different intrusion detection methods.

Furthermore, the number of records in the NSL-KDD train and test sets are reasonable. This advantage makes it affordable to run the experiments on the complete set without the need to randomly select a small portion. Consequently, evaluation results of different research work will be consistent and comparable.

Data files

* <spam style="color:green">  KDDTrain+.ARFF: The full NSL-KDD train set with binary labels in ARFF format </spam>
* KDDTrain+.TXT: The full NSL-KDD train set including attack-type labels and difficulty level in CSV format
* KDDTrain+_20Percent.ARFF: A 20% subset of the KDDTrain+.arff file
* KDDTrain+_20Percent.TXT: A 20% subset of the KDDTrain+.txt file
* KDDTest+.ARFF: The full NSL-KDD test set with binary labels in ARFF format
* KDDTest+.TXT: The full NSL-KDD test set including attack-type labels and difficulty level in CSV format
* KDDTest-21.ARFF: A subset of the KDDTest+.arff file which does not include records with difficulty level of 21 out of 21
* KDDTest-21.TXT: A subset of the KDDTest+.txt file which does not include records with difficulty level of 21 out of 21

[Link de Descarga](https://www.unb.ca/cic/datasets/nsl.html) 

License
You may redistribute, republish, and mirror the ISCX NSL-KDD dataset in any form. However, any use or redistribution of the data must include a citation to the NSL-KDD dataset and the paper referenced below.

References: [1] M. Tavallaee, E. Bagheri, W. Lu, and A. Ghorbani, “A Detailed Analysis of the KDD CUP 99 Data Set,” Submitted to Second IEEE Symposium on Computational Intelligence for Security and Defense Applications (CISDA), 2009.

## 1.- Lectura del DataSet  

In [1]:
import arff
import pandas as pd 

In [2]:
def load_kdd_dataset(data_path): 
    """Lectura del dataset NSL-KDD"""
    with open(data_path, 'r') as train_set:
        dataset = arff.load(train_set)
        attributes = [attr[0] for attr in dataset["attributes"]]
    return pd.DataFrame(dataset["data"], columns=attributes)

In [3]:
df = load_kdd_dataset("../datasets/NSL-KDD/KDDTrain+.arff")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125973 entries, 0 to 125972
Data columns (total 42 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   duration                     125973 non-null  float64
 1   protocol_type                125973 non-null  object 
 2   service                      125973 non-null  object 
 3   flag                         125973 non-null  object 
 4   src_bytes                    125973 non-null  float64
 5   dst_bytes                    125973 non-null  float64
 6   land                         125973 non-null  object 
 7   wrong_fragment               125973 non-null  float64
 8   urgent                       125973 non-null  float64
 9   hot                          125973 non-null  float64
 10  num_failed_logins            125973 non-null  float64
 11  logged_in                    125973 non-null  object 
 12  num_compromised              125973 non-null  float64
 13 

## Division del dataset 
Se debe separar el dataset en los diferentes subconjuntos necesarios para realizar los procesos de entrenamiento, validacion y pruebas. Sklearn tiene implemnetado la funcion **split_test_split**.

In [4]:
# Separamos el datset 60% train set, 40% test set
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(df, test_size=0.4, random_state=42)

In [5]:
train_set.info()

<class 'pandas.core.frame.DataFrame'>
Index: 75583 entries, 98320 to 121958
Data columns (total 42 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   duration                     75583 non-null  float64
 1   protocol_type                75583 non-null  object 
 2   service                      75583 non-null  object 
 3   flag                         75583 non-null  object 
 4   src_bytes                    75583 non-null  float64
 5   dst_bytes                    75583 non-null  float64
 6   land                         75583 non-null  object 
 7   wrong_fragment               75583 non-null  float64
 8   urgent                       75583 non-null  float64
 9   hot                          75583 non-null  float64
 10  num_failed_logins            75583 non-null  float64
 11  logged_in                    75583 non-null  object 
 12  num_compromised              75583 non-null  float64
 13  root_shell      

In [6]:
# Separar el datset de pruebas 50% para validation_set y 50% para test_set
val_set, test_set = train_test_split(test_set, test_size=0.5, random_state=42)

In [8]:
print("Longitud del Training Set:", len(train_set))
print("Longitud del Validation set:", len(val_set))
print("Longitud del Test set:", len(test_set))

Longitud del Training Set: 75583
Longitud del Validation set: 25195
Longitud del Test set: 25195


## 3.- Particionado aleatorio y Stratified Sampling

Sklearn tiene implemetada la funcion **train_test_split**, sin simbargo, esta funcion por defecto realiza un paticionado de datos aleatorio para cada vez que se ejecuta el script. Aun anadiendo una semilla fija para generacion aleatoria, cada vez que se cargue nuevamente el conjunto de datos se generaran nuevos subcojuntos. Esto puede ocasionar que despues de muchso intentos el algoritmo **"vea"** todo el  conjunto de datos. 

Para sloucionar este problema Sklearn ha introducido el parametro **shuffle** en la funcion **train_test_split**. 

In [10]:
# Si shuffle es = a false el dataset no mezclara antes del particionado 
train_set, test_set = train_test_split(df, test_size=0.4, random_state=42, shuffle=False)

Estos metodos para dividir el dataset estan bien si tienes un conjunto de datos muy grande, pero si no se tiene, se corre el riesgo de introducir **Sampling bias**.

Para evitar esto se utiliza un metodo de sampling que se llama **Stratified Sampling**. La poblacion es dividida en subconjuntos homogeneos llamados **strata**. El objetivo es que no quede ninguna caracteristica del conjunto de datos sin representacion en ninguno de los conjuntos de datos para una o mas caracteristicas en particular.

sklearn introduce el parametro **Stratify** en la funcion train_test_split para controlar este comportamiento. 

_This Stratify parameter makes a split so that the proportion of values in the sample produced will be the same as the proportion of values provided to paramete stratify._
_For example, if variable y is a binary categorical variable with values 0 and 1 there 25% of zeros 75% of ones, stratify = y will make sure that your random split has 25% of 0's and 75% 1's._

_https://stackoverflow.com_